In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

# Load the data
df = pd.read_csv('Winery_Data.csv')

# Clean data
df = df.dropna(subset=['Customer ID', 'Sale Amount', 'Year Acquired'])

<ipython-input-2-a685f331df82>:6: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Winery_Data.csv')


# CLV Analysis

In [4]:
# Convert date column to datetime
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')

# Get current year from the most recent transaction
current_year = df['Date'].max().year

# Calculate customer tenure in years
df['Tenure'] = current_year - df['Year Acquired']

# Ensure tenure is at least 1 year
df['Tenure'] = df['Tenure'].clip(lower=1)

# Group by customer
customer_data = df.groupby('Customer ID').agg({
    'Sale Amount': 'sum',
    'Order ID': 'count',
    'Tenure': 'max',
    'Year Acquired': 'min'
})

# Rename columns
customer_data.columns = ['Total_Sales', 'Total_Orders', 'Tenure', 'Acquisition_Year']

# Calculate average annual sales (simple CLV)
customer_data['Annual_Sales'] = customer_data['Total_Sales'] / customer_data['Tenure']

# Calculate average order value
customer_data['Avg_Order_Value'] = customer_data['Total_Sales'] / customer_data['Total_Orders']

# Calculate purchase frequency (orders per year)
customer_data['Purchase_Frequency'] = customer_data['Total_Orders'] / customer_data['Tenure']

# Basic CLV calculation (5-year projection)
discount_rate = 0.1  # 10% discount rate
projection_years = 5

# Projected CLV using simple formula: Annual_Sales * (1 - (1 + discount_rate)^(-projection_years)) / discount_rate
customer_data['5Y_CLV'] = customer_data['Annual_Sales'] * (1 - (1 + discount_rate)**(-projection_years)) / discount_rate

# Create CLV segments
customer_data['CLV_Segment'] = pd.qcut(customer_data['5Y_CLV'], 5,
                                       labels=['Low Value', 'Below Average', 'Average',
                                              'Above Average', 'High Value'])

# Display CLV segment distribution
clv_segments = customer_data['CLV_Segment'].value_counts()
print(clv_segments)

# Export CLV analysis
customer_data.to_csv('winery_clv_analysis.csv')

CLV_Segment
Average          4513
Low Value        4495
High Value       4433
Below Average    4380
Above Average    4365
Name: count, dtype: int64


# Purchase Behaviour Analysis

In [5]:
# Create a customer-level dataset with year-by-year sales
customer_behavior = df.groupby('Customer ID').agg({
    'Sales 2008': 'sum',
    'Sales 2009': 'sum',
    'Sales 2010': 'sum',
    'Sale Amount': 'sum',
    ' Orders 2008 ': 'sum',
    'Orders 2009': 'sum',
    'Orders 2010': 'sum'
})

# Calculate year-over-year growth rates
customer_behavior['Growth_2008_2009'] = ((customer_behavior['Sales 2009'] - customer_behavior['Sales 2008']) /
                                         customer_behavior['Sales 2008'].replace(0, np.nan)) * 100
customer_behavior['Growth_2009_2010'] = ((customer_behavior['Sales 2010'] - customer_behavior['Sales 2009']) /
                                         customer_behavior['Sales 2009'].replace(0, np.nan)) * 100

# Replace infinite values with NaN (happens when dividing by zero)
customer_behavior.replace([np.inf, -np.inf], np.nan, inplace=True)

# Define purchase trend segments
def purchase_trend(row):
    # Check if customer made purchases in all years
    active_all_years = (row['Sales 2008'] > 0) & (row['Sales 2009'] > 0) & (row['Sales 2010'] > 0)

    # Determine growth pattern
    if active_all_years:
        if (row['Growth_2008_2009'] > 10) & (row['Growth_2009_2010'] > 10):
            return 'Consistently Growing'
        elif (row['Growth_2008_2009'] < -10) & (row['Growth_2009_2010'] < -10):
            return 'Consistently Declining'
        elif (row['Growth_2008_2009'] > 10) & (row['Growth_2009_2010'] < -10):
            return 'Growth then Decline'
        elif (row['Growth_2008_2009'] < -10) & (row['Growth_2009_2010'] > 10):
            return 'Decline then Growth'
        else:
            return 'Stable'
    elif (row['Sales 2008'] == 0) & (row['Sales 2009'] > 0) & (row['Sales 2010'] > 0):
        return 'New in 2009'
    elif (row['Sales 2008'] == 0) & (row['Sales 2009'] == 0) & (row['Sales 2010'] > 0):
        return 'New in 2010'
    elif (row['Sales 2008'] > 0) & (row['Sales 2009'] > 0) & (row['Sales 2010'] == 0):
        return 'Churned in 2010'
    elif (row['Sales 2008'] > 0) & (row['Sales 2009'] == 0) & (row['Sales 2010'] == 0):
        return 'Churned in 2009'
    elif (row['Sales 2008'] > 0) & (row['Sales 2009'] == 0) & (row['Sales 2010'] > 0):
        return 'Reactivated in 2010'
    else:
        return 'Inconsistent'

customer_behavior['Purchase_Trend'] = customer_behavior.apply(purchase_trend, axis=1)

# Identify purchase channels
# Create binary flags for each channel
customer_behavior['Uses_Email'] = df.groupby('Customer ID')['Email Sales'].sum() > 0
customer_behavior['Uses_Newsletter'] = df.groupby('Customer ID')['Newsletter Sales'].sum() > 0
customer_behavior['Uses_Tasting_Room'] = df.groupby('Customer ID')['Tasting Room Sales'].sum() > 0
customer_behavior['Uses_Winemaker_Call'] = df.groupby('Customer ID')['Winemaker Call Sales'].sum() > 0

# Determine dominant channel
channels = ['Email Sales', 'Newsletter Sales', 'Tasting Room Sales', 'Winemaker Call Sales']
channel_sums = df.groupby('Customer ID')[channels].sum()

# Get the column name of the maximum sales channel for each customer
customer_behavior['Dominant_Channel'] = channel_sums.idxmax(axis=1).apply(lambda x: x.replace(' Sales', ''))

# Display purchase trend distribution
trend_counts = customer_behavior['Purchase_Trend'].value_counts()
print(trend_counts)

# Display dominant channel distribution
channel_counts = customer_behavior['Dominant_Channel'].value_counts()
print(channel_counts)

# Export purchase behavior analysis
customer_behavior.to_csv('winery_purchase_behavior.csv')

Purchase_Trend
Inconsistent              7508
Churned in 2010           6734
New in 2009               3335
New in 2010               1367
Churned in 2009            771
Growth then Decline        698
Consistently Declining     529
Decline then Growth        502
Stable                     345
Reactivated in 2010        211
Consistently Growing       186
Name: count, dtype: int64
Dominant_Channel
Tasting Room      18868
Email              1623
Newsletter         1606
Winemaker Call       89
Name: count, dtype: int64


# RFM Analysis

In [6]:
# Convert date to datetime format - fix the format to match your data
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')

# Calculate RFM metrics
# Get the most recent date in the dataset
max_date = df['Date'].max()

# Group by customer
rfm = df.groupby('Customer ID').agg({
    'Date': lambda x: (max_date - x.max()).days,  # Recency
    'Order ID': 'count',  # Frequency
    'Sale Amount': 'sum'  # Monetary
})

# Rename columns
rfm.columns = ['Recency', 'Frequency', 'Monetary']

# Fix for duplicate bin edges - use qcut with duplicates='drop'
# Also handle case where we might have too many identical values
try:
    rfm['R_Quartile'] = pd.qcut(rfm['Recency'], 4, labels=[4, 3, 2, 1], duplicates='drop')
except ValueError:
    # If too many identical values, use rank method instead
    rfm['R_Quartile'] = pd.cut(rfm['Recency'].rank(method='first'),
                              bins=4, labels=[4, 3, 2, 1])

try:
    rfm['F_Quartile'] = pd.qcut(rfm['Frequency'], 4, labels=[1, 2, 3, 4], duplicates='drop')
except ValueError:
    # If too many identical values, use rank method instead
    rfm['F_Quartile'] = pd.cut(rfm['Frequency'].rank(method='first'),
                              bins=4, labels=[1, 2, 3, 4])

try:
    rfm['M_Quartile'] = pd.qcut(rfm['Monetary'], 4, labels=[1, 2, 3, 4], duplicates='drop')
except ValueError:
    # If too many identical values, use rank method instead
    rfm['M_Quartile'] = pd.cut(rfm['Monetary'].rank(method='first'),
                              bins=4, labels=[1, 2, 3, 4])

# Calculate RFM Score
rfm['RFM_Score'] = rfm['R_Quartile'].astype(str) + rfm['F_Quartile'].astype(str) + rfm['M_Quartile'].astype(str)

# Create RFM Segments
def rfm_segment(rfm_score):
    if rfm_score in ['444', '443', '434', '344']:
        return 'Champions'
    elif rfm_score in ['442', '432', '342', '332', '341', '431']:
        return 'Loyal Customers'
    elif rfm_score in ['433', '343', '333', '323', '422']:
        return 'Potential Loyalists'
    elif rfm_score in ['334', '324', '234', '233', '232', '231', '224', '223', '222']:
        return 'New Customers'
    elif rfm_score in ['413', '414', '313', '314', '423', '424']:
        return 'Promising'
    elif rfm_score in ['331', '321', '312', '311', '221', '212']:
        return 'Need Attention'
    elif rfm_score in ['134', '133', '132', '124', '123', '122', '121']:
        return 'About to Sleep'
    elif rfm_score in ['144', '143', '142', '141', '241', '251', '252', '253']:
        return 'At Risk'
    elif rfm_score in ['114', '113', '112', '111', '213', '212', '211']:
        return 'Cannot Lose Them'
    else:
        return 'Hibernating'

rfm['RFM_Segment'] = rfm['RFM_Score'].apply(rfm_segment)

# Display segment distribution
segment_counts = rfm['RFM_Segment'].value_counts()
print(segment_counts)

# Export the RFM segments
rfm.to_csv('winery_rfm_segments.csv')

RFM_Segment
Champions              3474
Need Attention         3207
Cannot Lose Them       2704
Hibernating            2599
About to Sleep         2569
Potential Loyalists    2320
New Customers          1993
Loyal Customers        1474
Promising              1346
At Risk                 500
Name: count, dtype: int64
